# Discovering MlflowClient API 

- Create experiment
- List experiments
- Discovering experiment runs
- Manipulating runs info and data
- Model register
- Model versioning
- Testing models as a deployment engineer 
- Modify model alieses 

In [1]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

2025/06/20 08:58:57 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/20 08:58:57 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

### Experiments

In [2]:
experiment_tags = {
  "developer": "kamal",
  "data": "nyc-taxi"
}

# created new experiment
# articat location parameter if None defaults to "mlfurn/{experiment_id}"
client.create_experiment("nyc-taxi-experiment", tags=experiment_tags)

'1'

In [3]:
# dicovering existing experiment
existing_experiments = client.search_experiments()

print(f"There exist {len(existing_experiments)} experiments")
for exp in sorted(existing_experiments, key=lambda exp: exp.experiment_id):
  print(f"{exp.experiment_id}: {exp.name}")
  for key, value in exp.tags.items():
    print(f"\t{key}: {value}")

There exist 2 experiments
0: Default
1: nyc-taxi-experiment
	developer: kamal
	data: nyc-taxi


### Creating some Runs

In [4]:
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction import DictVectorizer

from data_loader import read_dataframe

In [5]:
training_data = ["../data/green_tripdata_2021-01.parquet"]
validation_data = ["../data/green_tripdata_2021-03.parquet"]
test_data = ["../data/green_tripdata_2021-02.parquet"]

In [6]:
df_train = read_dataframe(*training_data)
df_validation = read_dataframe(*validation_data)

df_train.shape, df_validation.shape

((73908, 3), (80372, 3))

In [7]:
features = ['PU_DO', 'trip_distance']
target = 'duration'

dv = DictVectorizer()

X_train = dv.fit_transform(df_train[features].to_dict(orient="records"))
X_validation = dv.transform(df_validation[features].to_dict(orient="records"))

y_train = df_train[target]
y_validation = df_validation[target]

In [8]:
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("sqlite:///mlflow.db")
# This method should create the experiment if not exists
# But I created the experiment with same name on the prev step so mlflow will just use it
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

mlflow.sklearn.autolog()

In [9]:
sklearn_models = [
  LinearRegression(), 
  Lasso(alpha=0.01, random_state=42),
  DecisionTreeRegressor(max_depth=3, random_state=42),
  RandomForestRegressor(n_estimators=5, max_depth=3, random_state=42)
]

runs: dict = []

In [10]:
from tqdm import tqdm

for model in tqdm(sklearn_models):
  model_name = str(model)
  model_name = model_name[:model_name.index("(")]
  with mlflow.start_run(run_name=model_name) as current_run:
    lr = model.fit(X_train, y_train)
    runs.append({"id": current_run.info.run_id, "name": current_run.info.run_name})  

100%|██████████| 4/4 [02:00<00:00, 30.12s/it]


In [11]:
for run in runs:
  print(f"{run['id']}: {run['name']}")

a8f3bd171b9545999c3900dfb2b3ea53: LinearRegression
75604984f6b34d528b7220f1566ee5a6: Lasso
6b507b9892a64d938bdd32dd5349ccd0: DecisionTreeRegressor
7dcda490536c47509575fc052ccd4603: RandomForestRegressor


### Discovering runs

In [12]:
retrieved_runs = client.search_runs(
  experiment_ids="1",
  filter_string="",
  run_view_type=ViewType.ACTIVE_ONLY,
  order_by=["metrics.training_root_mean_squared_error ASC"]
)

assert len(retrieved_runs) == len(runs)
for run in retrieved_runs:
  assert run.info.run_id in list(map(lambda run: run["id"], runs))

In [13]:
for run in retrieved_runs:
  print(f"{run.info.run_name}: {run.data.metrics['training_root_mean_squared_error']}")

LinearRegression: 5.6995641181989996
RandomForestRegressor: 6.293870290580047
DecisionTreeRegressor: 6.315541694545283
Lasso: 10.550479041655857


In [14]:
# get run by condition
for run in client.search_runs(experiment_ids="1", filter_string="metrics.training_root_mean_squared_error < 7", order_by=["metrics.training_root_mean_squared_error ASC"]):
  print(f"{run.info.run_name}: {run.data.metrics['training_root_mean_squared_error']}")

LinearRegression: 5.6995641181989996
RandomForestRegressor: 6.293870290580047
DecisionTreeRegressor: 6.315541694545283


In [23]:
mlflow.search_runs(experiment_ids="1", filter_string="metrics.training_root_mean_squared_error < 7", order_by=["metrics.training_root_mean_squared_error ASC"])[["run_id", "artifact_uri", "metrics.training_root_mean_squared_error"]]

,run_id,artifact_uri,metrics.training_root_mean_squared_error
0,a8f3bd171b9545999c3900dfb2b3ea53,/Users/Kamal/WorkSpace/MLOps-zoomcamp/02. Expe...,5.699564
1,7dcda490536c47509575fc052ccd4603,/Users/Kamal/WorkSpace/MLOps-zoomcamp/02. Expe...,6.293870
2,6b507b9892a64d938bdd32dd5349ccd0,/Users/Kamal/WorkSpace/MLOps-zoomcamp/02. Expe...,6.315542


### Staging models

After discovering the deciding the best models, now you can register some models to the model_registery.
That's where the ML engineer or the deployment engineer testing models and choosing which model to deploy

data scientest => work till ML models staging \
deployment engineer => work on the staging models and decides which to deploy

In [24]:
REGISTERED_MODEL_NAME = "nyc-taxi-model"

#### Note

There are 2 methods to create a containing registry model:
1. using mlflow.register_model(mode_uri, containing_model_name) \
  This method will create the containing model if not exists
2. using client.create_registered_mode(containing_model_name) \
   using client.create_model_version(containing_model_name, model_uri, version_tags, description) \
   This method gives more control over the creation and maintaining of the model and their versions

In [32]:
mlflow.register_model(model_uri="runs:/85aae2e0d958479ba524144afc5fc0b3/model", name=REGISTERED_MODEL_NAME)

2025/06/14 14:22:10 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/14 14:22:10 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'nyc-taxi-model'.
2025/06/14 14:22:10 WARNING mlflow.tracking._model_registry.fluent: Run with id 85aae2e0d958479ba524144afc5fc0b3 has no artifacts at artifact path 'model', registering model based on models:/m-71c170708850403d9e5fd0df7cb709b5 instead
Created version '1' of model 'nyc-taxi-model'.


<ModelVersion: aliases=[], creation_timestamp=1749900130753, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1749900130753, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='85aae2e0d958479ba524144afc5fc0b3', run_link=None, source='models:/m-71c170708850403d9e5fd0df7cb709b5', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [25]:
from datetime import date

client.create_registered_model(
  name=REGISTERED_MODEL_NAME,
  tags={
    "creator": "kamal",
    "problem": "nyc-taxi",
  },
  description=f"created at {date.today()}"
)

2025/06/20 09:06:20 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/06/20 09:06:20 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<RegisteredModel: aliases={}, creation_timestamp=1750399580880, deployment_job_id=None, deployment_job_state=None, description='created at 2025-06-20', last_updated_timestamp=1750399580880, latest_versions=[], name='nyc-taxi-model', tags={'creator': 'kamal', 'problem': 'nyc-taxi'}>

In [26]:
for run in retrieved_runs:
  client.create_model_version(
    name = REGISTERED_MODEL_NAME,
    source=f"runs:/{run.info.run_id}/model",
    tags={"name": f"{run.info.run_name}"},
    description=f"Moved to registry on {date.today()}"
  )
  
for version in client.search_model_versions(filter_string=f"name = '{REGISTERED_MODEL_NAME}'"):
  print(f"version:{version.version}, URI: {client.get_model_version_download_uri(name=REGISTERED_MODEL_NAME, version=version.version)}")

version:4, URI: runs:/75604984f6b34d528b7220f1566ee5a6/model
version:3, URI: runs:/6b507b9892a64d938bdd32dd5349ccd0/model
version:2, URI: runs:/7dcda490536c47509575fc052ccd4603/model
version:1, URI: runs:/a8f3bd171b9545999c3900dfb2b3ea53/model


In [27]:
client.set_registered_model_alias(REGISTERED_MODEL_NAME, "waiting-release", "4")

### ML engineer testing the models to promote good ones

In [28]:
df_test = read_dataframe(test_data)

X_test = dv.transform(df_test[features].to_dict(orient="records"))
y_test = df_test[target]

In [29]:
client.get_model_version_download_uri(REGISTERED_MODEL_NAME, "1")

'runs:/a8f3bd171b9545999c3900dfb2b3ea53/model'

In [31]:
import mlflow.sklearn

testing_results = {}
for version in client.search_model_versions(filter_string=f"name = '{REGISTERED_MODEL_NAME}'"): 
  model_uri = client.get_model_version_download_uri(REGISTERED_MODEL_NAME, version.version)
  model = mlflow.sklearn.load_model(model_uri)
  y_test_pred = model.predict(X_test)

  testing_results[version.version] = root_mean_squared_error(y_test, y_test_pred)

In [32]:
for res in sorted(testing_results.items(), key=lambda item: item[1]):
  print(f"version: {res[0]}: {res[1]:.04f}")

version: 2: 7.0707
version: 3: 7.0890
version: 1: 7.7587
version: 4: 11.1673


In [35]:
# According to the results, we can inspect that version 2 is the best model so far
for alias, model_version in client.get_registered_model(name = REGISTERED_MODEL_NAME).aliases.items():
  print(f"{model_version}: {alias}")

2: deployment
4: waiting-release


In [34]:
# now update the best model to set in production
client.delete_registered_model_alias(name=REGISTERED_MODEL_NAME, alias="rejected_need-refinement")
client.set_registered_model_alias(REGISTERED_MODEL_NAME, "deployment", "2")